# Imports

In [ ]:
import pandas as pd
from datetime import datetime as dt
from hbt_tools import messdatenbank as mdb
from pathlib import Path
from plotly import graph_objects as go
from plotly.subplots import make_subplots

# Parameters

In [ ]:
dir_path = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\8 Entwässerungskonzept\SIMBA Modell\06_Auswertungsskript_Visualisierung")

# Read Data

In [ ]:
file_path = dir_path / 'Entlastungskennzahlen-SBW_2023-10-26.xlsx'
date_format = '%Y-%m-%d %H:%M:%S'
data = pd.read_excel(file_path, index_col=0, parse_dates=[0], date_format=date_format, skiprows=[1])

In [ ]:
id = 1780
date_from = dt(2019, 9, 21)
date_to = dt(2023, 10, 14)
rain = mdb.get_data(id, date_from, date_to)

In [ ]:
rain['kumulativ [mm]'] = rain.cumsum()

# Plots

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
for name in data.columns:
    fig.add_trace(go.Scatter(x=data.index, y=data[name], name=name))

fig.add_trace(go.Scatter(x=rain.index, y=rain.iloc[:,1], name='Regen WAE [mm]',
    line=dict(color='blue', dash='dash')), secondary_y=True, )

fig.update_layout(title='Entlastungsdauer SBW')
fig.update_xaxes(title='Zeit')
fig.update_yaxes(title='Dauer Entlastung [min]')

save_path = dir_path / 'Entlastungsdauer-SBW.html'
fig.write_html(save_path)

# Analysis

In [ ]:
sum_entlastungen = data.groupby(pd.Grouper(freq='Y')).sum() / 60
sum_entlastungen